In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import GridSearchCV
import shap

from xgboost import XGBClassifier

In [2]:
df = pd.read_csv('mimic_los_subset.csv')

In [5]:
X = df.drop(['actualLOS'], axis=1)


In [8]:
y = df['actualLOS']
y_arr = np.asarray(df['actualLOS'], dtype="|S6")

In [9]:
clf_etc = ExtraTreesClassifier()
clf_etc.fit(X,y_arr)

/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [10]:
feat_importances = pd.Series(clf_etc.feature_importances_, index=X.columns)
feature_list = feat_importances.nlargest(20).index.tolist()

In [11]:
df = df[feature_list]
X = df

In [12]:
# Create training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=7)

In [13]:
num_prediction_rows = 5
data_to_predict = X_train.iloc[0:num_prediction_rows]

In [14]:
xgb = XGBClassifier(silent=False, learning_rate=0.01, n_estimators=5, max_depth=2, subsample=0.8, colsample_bytree=1, gamma=1)
xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=1, learning_rate=0.01, max_delta_step=0,
       max_depth=2, min_child_weight=1, missing=None, n_estimators=5,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=False, subsample=0.8)

In [15]:
explainer = shap.TreeExplainer(xgb)
shap_values = explainer.shap_values(data_to_predict)

In [ ]:
shap.summary_plot(shap_values, data_to_predict)